In [1]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


rng = 69

In [2]:
#df = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Transformed.csv")
df_train = pd.read_csv(r"C:\Users\ZaneC\OneDrive\Documents\Python\Kaggle\Playground Series\s4e12 - Insurance Premiums\train_transformed.csv")
df_test = pd.read_csv(r"C:\Users\ZaneC\OneDrive\Documents\Python\Kaggle\Playground Series\s4e12 - Insurance Premiums\test_transformed.csv")

In [5]:
# Heatmap to visualize correlations
#corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [6]:
#corr_matrix['Premium Amount'].sort_values(ascending=False)

Premium Amount                 1.000000
Previous Claims                0.041870
Customer Feedback_Unknown      0.028580
Health Score                   0.015033
Policy Start Month             0.007620
Policy Start Month cos         0.005139
Age Category_29-37             0.002780
Occupation_Self-Employed       0.002268
Occupation_Unemployed          0.001413
Location_Urban                 0.001409
Age Category_38-46             0.001329
Education Level_High School    0.001323
Marital Status_Single          0.000966
Vehicle Age                    0.000514
Policy Start Weekday           0.000185
Policy Type_Comprehensive      0.000044
Exercise Frequency_Monthly    -0.000048
Unnamed: 0                    -0.000051
id                            -0.000051
Smoking Status_Yes            -0.000181
Unnamed: 42                   -0.000195
Insurance Duration            -0.000195
Gender_Male                   -0.000271
Education Level_Master's      -0.000299
Exercise Frequency_Weekly     -0.000440


In [35]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [12]:
# Set train data
X_train = df_train.drop('Premium Amount', axis=1)
Y_train = df_train['Premium Amount'].copy()

X_valid = df_train.drop('Premium Amount', axis=1)
Y_valid = df_train['Premium Amount'].copy()

X_test = df_test

20.0

In [13]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [14]:
from keras import backend as K

#Define a custom RMSLE metric
def rmsle(y_true, y_pred):
    # To avoid log(0), add a small constant (epsilon)
    epsilon = tf.keras.backend.epsilon()
    
    # Clip values to avoid log(0)
    y_true = tf.clip_by_value(y_true, epsilon, None)  # Ensure no zero values for y_true
    y_pred = tf.clip_by_value(y_pred, epsilon, None)  # Ensure no zero values for y_pred
    
    # RMSLE formula
    return tf.sqrt(tf.reduce_mean(tf.square(tf.math.log(y_pred + epsilon) - tf.math.log(y_true + epsilon))))

In [9]:
n_epoch = 25

In [15]:
def build_model(n_neurons=50, n_hidden=3, dropout_rate=0.4, learning_rate=1e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    # Hidden layer with droupout
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu'))
        model.add(keras.layers.Dropout(dropout_rate))
    # Output layer    
    model.add(keras.layers.Dense(1))
    
    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,loss='mean_squared_logarithmic_error',metrics=['msle'])
    return model

In [16]:
from scikeras.wrappers import KerasRegressor

# Seems like you need to specify some items again when using the scikeras wrapper
keras_reg = KerasRegressor(model=build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)


In [17]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'model__n_hidden': [0,1,2,3,4,5],
    'model__dropout_rate': [0,.1,.2,.3,.4],
    'model__n_neurons': np.arange(10,60),
    'model__learning_rate' : reciprocal(1e-5, 1e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=5, cv=3, scoring='neg_root_mean_squared_log_error',
                                   error_score = 'raise')

early_stopping_cb = keras.callbacks.EarlyStopping(patience = 7, restore_best_weights = True)
rnd_search_cv.fit(X_train_prepared,Y_train, epochs=n_epoch,
                 validation_data = (X_valid_prepared, Y_valid),
                 callbacks=[early_stopping_cb])

Epoch 1/25
22729/22729 ━━━━━━━━━━━━━━━━━━━━ 31s 1ms/step - loss: 28.0850 - msle: 28.0850 - val_loss: 10.8333 - val_msle: 10.8333
Epoch 2/25
22729/22729 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - loss: 3.9492 - msle: 3.9492 - val_loss: 1.7206 - val_msle: 1.7206
Epoch 3/25
22729/22729 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - loss: 1.2988 - msle: 1.2988 - val_loss: 1.2443 - val_msle: 1.2443
Epoch 4/25
22729/22729 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - loss: 1.2515 - msle: 1.2515 - val_loss: 1.2083 - val_msle: 1.2083
Epoch 5/25
22729/22729 ━━━━━━━━━━━━━━━━━━━━ 41s 1ms/step - loss: 1.2395 - msle: 1.2395 - val_loss: 1.2014 - val_msle: 1.2014
Epoch 6/25
22729/22729 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - loss: 1.2416 - msle: 1.2416 - val_loss: 1.1997 - val_msle: 1.1997
Epoch 7/25
22729/22729 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - loss: 1.2377 - msle: 1.2377 - val_loss: 1.1991 - val_msle: 1.1991
Epoch 8/25
22729/22729 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 1.2403 - msle: 1.2403 - val_loss: 1.1987 - val_msle: 1.1

ValueError: Root Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [43]:
#print(rnd_search_cv.best_params_)
#print(rnd_search_cv.best_score_)

In [44]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

{'model__dropout_rate': 0.4, 'model__learning_rate': 0.001157866133544249, 'model__n_hidden': 2, 'model__n_neurons': 25}
-5.5286867306164025


In [45]:
best_model = rnd_search_cv.best_estimator_.model

In [46]:
from scikeras.wrappers import KerasRegressor

# Seems like you need to specify some items again when using the scikeras wrapper
keras_reg = KerasRegressor(model=best_model, epochs=n_epoch, batch_size=5, verbose=0, random_state=42,
                           loss='mean_squared_error',optimizer='adam',metrics=['mae'])

# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
#checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)

keras_reg.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid),
             callbacks=[early_stopping_cb])


KerasRegressor(
	model=<function build_model at 0x000002A021429A80>
	build_fn=None
	warm_start=False
	random_state=42
	optimizer=adam
	loss=mean_squared_error
	metrics=['mae']
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=150
)

In [47]:
from sklearn.model_selection import cross_val_score

# Assuming X and y are your feature matrix and target vector
scores = cross_val_score(keras_reg, X_train_prepared, Y_train, cv=5, scoring='neg_mean_squared_error',verbose=0)

# Convert scores to positive values
mse_scores = -scores
print(f'MSE Scores: {mse_scores}')
print(f'Mean MSE: {np.mean(mse_scores)}')

MSE Scores: [6.02940222 4.9634603  5.30375332 5.54431183 5.85688376]
Mean MSE: 5.539562287606165


In [48]:
X_train_pred = keras_reg.predict(X_train_prepared)
X_test_pred = keras_reg.predict(X_test_prepared)

# Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(Y_train, X_train_pred)
mae = mean_absolute_error(Y_train, X_train_pred)

print(f'Train MSE: {mse}') # Sensitive to outliers
print(f'Train MAE: {mae}')

mse = mean_squared_error(Y_test, X_test_pred)
mae = mean_absolute_error(Y_test, X_test_pred)

print(f'Test MSE: {mse}') # Sensitive to outliers
print(f'Test MAE: {mae}')

Train MSE: 5.344163711089202
Train MAE: 1.8470409416088096
Test MSE: 5.315119719057888
Test MAE: 1.879280097918077


In [49]:
#df_Predict = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Predict.csv")
df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Over_Under_Predict.csv")

In [50]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T

y_pred = np.expm1(model.predict(X_test))  # Inverse of log1p (to get the original scale)

NameError: name 'keras_clf' is not defined